In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import tensorflow as tf

In [2]:
# Note: The format of data is .npy, which means numpy arrays.
DATA_PATH = os.path.join(r"C:\Users\Flavia Saldanha\Downloads\Data")

# Actions that we try to detect
actions = np.array(["HI","THANKS","FISH","COW","PLANE","CYCLING"])

no_sequences = 10

sequence_length = 20

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [5]:
label_map

{'HI': 0, 'THANKS': 1, 'FISH': 2, 'COW': 3, 'PLANE': 4, 'CYCLING': 5}

In [6]:
np.array(sequences).shape

(60, 20, 258)

In [7]:
X = np.array(sequences)

In [8]:
y = to_categorical(labels).astype(int)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
y_test.shape

(18, 6)

# Modelling

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [12]:
# Designing Neural Network
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

C:\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Compiling
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [14]:
# Fitting
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - categorical_accuracy: 0.1791 - loss: 1.7662
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.2946 - loss: 1.6734
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - categorical_accuracy: 0.2212 - loss: 1.5276
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.3631 - loss: 1.4040
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - categorical_accuracy: 0.3844 - loss: 1.1658
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.4474 - loss: 1.0951
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.4216 - loss: 0.9899
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - categorical_accuracy: 0.8209 - loss: 0.8177
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.7738 - loss: 0.5681
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - categorical_accuracy: 0.6528 - loss: 0.7892
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

In [15]:
# Summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20, 64)         │        82,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 20, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 712,052 (2.72 MB)

 Trainable params: 237,350 (927.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 474,702 (1.81 MB)

# Evaluation of model on test data

In [16]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step


In [17]:
# Saving the trained Model
model.save('isl6.h5')

In [18]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [19]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [20]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[14,  0],
        [ 1,  3]],

       [[13,  2],
        [ 0,  3]],

       [[13,  3],
        [ 0,  2]],

       [[17,  0],
        [ 0,  1]],

       [[14,  1],
        [ 1,  2]],

       [[13,  0],
        [ 4,  1]]], dtype=int64)

In [21]:
accuracy_score(ytrue, yhat)

0.6666666666666666

In [22]:
from sklearn.metrics import precision_score, recall_score
precision_score(ytrue, yhat,average='macro')

0.7777777777777777

In [23]:
recall_score(ytrue, yhat,average='macro')

0.7694444444444445

In [24]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss: {}, Test Accuracy: {}'.format(loss, accuracy))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - categorical_accuracy: 0.6667 - loss: 0.5378
Test Loss: 0.5378146171569824, Test Accuracy: 0.6666666865348816
